In [1]:
# Imports
from Gibbs.GibbsSampler import GibbsSampler
from Gibbs.Parameters import *
from DataGeneration import generate_multiple_series
import Gibbs.Initialize as Initialize
import Gibbs.Priors as Priors
import numpy as np
from Trends import Regressors
from Trends import LowFrequencyTrends
import torch

## Load Data
Load data and populations weights for the Gibbs sampler

In [2]:
data = generate_multiple_series(150, 120, 10**4, 10**5, "gibbs_data.csv")
q = 16
q_hat = 31
n = len(data)
m = 25
l = 10
T = len(data[0])
w = np.random.random(n)
R_hat = Regressors.find_regressors(T, q_hat).T
# Normalize the weights to sum to 1
w = w/np.sum(w)
print('Sum of weights: '+str(np.sum(w)))
print('Dimensions of R_hat: '+str(R_hat.shape))

Sum of weights: 1.0
Dimensions of R_hat: (150, 32)


## Initialize the Gibbs Sampler

### Step-by-step initialization

Initialize $X_i$:

In [3]:
X_i = Initialize.initialize_X(data, q)
print("# Entries in X_i: "+str(len(X_i)))
print("Length of each coefficient vector: "+str(len(X_i[0])))

# Entries in X_i: 120
Length of each coefficient vector: 32


Initialize $(F, S_m)$:

In [4]:
F = Initialize.initialize_F(X_i, w, q_hat)
print("Length of F: "+str(len(F)))

Length of F: 32


In [5]:
S_m, sigma_m, Sigma_m = Initialize.initialize_S_m(T, R_hat)
print("Length of S_m: "+str(len(S_m)))

Length of S_m: 32


Initialize $\{K(j)\}_{j=1}^{25}$ and $\{J(i)\}_{i=1}^n$:

In [6]:
K = Priors.group_factors(m, l)
J = Priors.group_factors(n, m) 
print("K has "+str(len(K))+" factors")
print("J has "+str(len(J))+" factors")

K has 25 factors
J has 120 factors


Initalize $\{\lambda_{c,i}\}_{i=1}^n$ and $\{\lambda_{g,j}\}_{j=1}^{25}$ parameters:

In [7]:
lambdas = Lambda_Parameters(n, m)
print("We have "+ str(len(lambdas.lambda_c_i))+" lambda_c_i parameters")
print("We have "+ str(len(lambdas.lambda_g_j))+" lambda_g_j parameters")

We have 120 lambda_c_i parameters
We have 25 lambda_g_j parameters


Initalize $\{\kappa_{c,i}\}_{i=1}^n$, $\{\kappa_{g,j}\}_{j=1}^{25}$ 
and $\{\kappa_{h,k}\}_{k=1}^{10}$ parameters:

In [8]:
kappas = Kappa_Parameters(n,m,l)
print("We have "+ str(len(kappas.kappa_c_i))+" kappa_c_i parameters")
print("We have "+ str(len(kappas.kappa_g_j))+" kappa_g_j parameters")
print("We have "+ str(len(kappas.kappa_h_k))+" kappa_h_k parameters")

We have 120 kappa_c_i parameters
We have 25 kappa_g_j parameters
We have 10 kappa_h_k parameters


Initialize all $p$ parameters

In [9]:
p_parameters = P_Parameters()
print("We have "+ str(len(p_parameters.p_c_lambda))+" p_c_l parameters")
print("We have "+ str(len(p_parameters.p_g_lambda))+" p_g_l parameters")
print("We have "+ str(len(p_parameters.p_c_theta))+" p_c_theta parameters")
print("We have "+ str(len(p_parameters.p_g_theta))+" p_g_theta parameters")
print("We have "+ str(len(p_parameters.p_h_theta))+" p_h_theta parameters")
print("We have "+ str(len(p_parameters.p_c_kappa))+" p_c_k parameters")
print("We have "+ str(len(p_parameters.p_g_kappa))+" p_g_k parameters")
print("We have "+ str(len(p_parameters.p_h_k))+" p_h_k parameters")

We have 25 p_c_l parameters
We have 25 p_g_l parameters
We have 100 p_c_theta parameters
We have 100 p_g_theta parameters
We have 100 p_h_theta parameters
We have 25 p_c_k parameters
We have 25 p_g_k parameters
We have 25 p_h_k parameters


Initialize $\sigma_{\Delta a}^2$:

In [10]:
s_Da = Initialize.inverse_squared(0.03**2)
print("s_Da = "+str(s_Da))

s_Da = 4.250042500425004


Initialize $f_0$, $\mu_m$ and $\mu_c$:

In [11]:
f_0 = Priors.flat_prior(0, 10**6)
mu_m = Priors.flat_prior(0, 10**6)
mu_c = Priors.flat_prior(0, 10**6)
print("f_0 = "+str(f_0))
print("mu_m = "+str(mu_m))
print("mu_c = "+str(mu_c))

f_0 = 561651.4730103255
mu_m = 738791.691688194
mu_c = 967073.662156095


Initialize $\omega^2$:

In [12]:
omega_squared = Initialize.inverse_squared(1)
print("omega_squared = " + str(omega_squared))

omega_squared = 61.230612306123064


Initialize $U_{c,i}$, $U_{g,j}$ and $U_{h,k}$:

In [13]:
us = U_Parameters(omega_squared, lambdas, kappas, R_hat, n, m, l, T)

c:\Users\konto\Documents\Economy-and-Population-Growth\Gibbs\Priors.py:78: RuntimeWarning: covariance is not positive-semidefinite.
  return np.random.multivariate_normal(mean=mean, cov=cov, size=n)


In [14]:
print(len(us.U_c))
print(len(us.U_c[0]))
print(len(us.U_g))
print(len(us.U_g[0]))
print(len(us.H))
print(len(us.H[0]))

120
32
25
32
10
32


In [15]:
G = []
for j in range(m):
    G_j = lambdas.lambda_g_j[j]*us.H[K[j]]+us.U_g[j]
    G.append(G_j)

C = []
i_1 = np.zeros(q_hat+1)
i_1[0] = 1
for i in range(n):
    C_i = mu_c*i_1+lambdas.lambda_c_i[i]*G[J[i]]+us.U_c[i]
    C.append(C_i)
G = np.array(G)
C = np.array(C)
print(len(G))
print(len(C))

25
120


## Steps of a Gibbs Draw

### Step 1: Calculate $\{X_i\}_{i=1}^n$

Calculate $Y^0$, $\mu_C$, $\Sigma$, $\Beta$, $w$, $m$ and $V$:

In [16]:
Y0 = w@C

In [17]:
i_1 = np.zeros(q_hat+1)
i_1[0] = 1
mu_C = np.concatenate([mu_c*i_1.T+lambdas.lambda_c_i[i]*G[J[i]].T for i in range(n)]).T

In [18]:
dim = (q_hat+1)*n
Sigma = np.zeros((dim, dim))
Sigma[0:(q_hat+1), 0:(q_hat+1)] = us.S_U_c[0]
for i in range(n):
    Sigma[i*(q_hat+1):(i+1)*(q_hat+1), i*(q_hat+1):(i+1)*(q_hat+1)] = us.S_U_c[i]

In [19]:
# X = np.concatenate([X_i[i].T for i in range(n)]).T
# X = np.reshape(X_i, (dim, ))
X = X_i.flatten()
Ys = []
for i in range(n):
    Y_i, _ = LowFrequencyTrends.find_trends(data[i], q)
    Ys.append(Y_i)

Y = np.concatenate([Ys[i] for i in range(len(Ys))])

def find_B(A, C):
    a = len(A)
    c = len(C)
    B = np.zeros((a, c))
    for i in range(a):
        B[i, :] = A[i]*C
    return B.T

B = find_B(Y, X)
I = np.identity(q_hat+1)
ws = np.kron(w.T, I).T
print(B.shape)

(3840, 2040)


In [20]:
Cs = np.concatenate([C[i] for i in range(len(C))])

In [21]:
help = Sigma@B
mm = help@np.linalg.inv(B.T@help)@B.T@(Cs-mu_C)

In [22]:
V = Sigma-help@np.linalg.inv(B.T@help)@B.T@Sigma
print(V.shape)

(3840, 3840)


In [23]:
help@np.linalg.inv(B.T@help)@B.T@Sigma

array([[-1.04397305e+02, -8.13758095e-01, -5.17805601e+02, ...,
         3.71673044e-02, -5.63409197e+00, -3.24447641e-02],
       [-4.13548496e-01, -3.22353566e-03, -2.05118060e+00, ...,
         1.47230647e-04, -2.23182989e-02, -1.28523273e-04],
       [-1.84045745e+02, -1.43460327e+00, -9.12858023e+02, ...,
         6.55235709e-02, -9.93254236e+00, -5.71980357e-02],
       ...,
       [ 2.04286176e-02,  1.59237376e-04,  1.01324958e-01, ...,
        -7.27295255e-06,  1.10248737e-03,  6.34883896e-06],
       [-3.16772362e+00, -2.46918321e-02, -1.57117564e+01, ...,
         1.12776616e-03, -1.70955047e-01, -9.84470292e-04],
       [-2.60707521e-02, -2.03216793e-04, -1.29309673e-01, ...,
         9.28165315e-06, -1.40698090e-03, -8.10231068e-06]])

In [24]:
inv1 = np.linalg.inv(B.T@help)

In [25]:
mat = torch.from_numpy(B.T@help)

In [26]:
inv2 = torch.inverse(mat)

In [27]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    B = torch.tensor(B, device=device)
    help = torch.tensor(help, device=device)
    Sigma = torch.tensor(Sigma, device=device)
    inv2 = torch.inverse(torch.matmul(B.t(), help))
    mat = torch.chain_matmul(help, inv2, B.t(), Sigma)
    result = torch.inverse(mat)
    result = result.to("cpu")

Draw $Z_0$, calculate $Z_1$, draw $\epsilon$ and caclulate $C$:

In [28]:
Z0 = Priors.multivariate_normal_prior(mu_C, V/np.linalg.norm(V))[0]

In [29]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
normal_V = V/np.linalg.norm(V)
mu_C = torch.tensor(mu_C, device=device)
normal_V = torch.tensor(normal_V, device=device)

In [30]:
def make_positive_definite(matrix, epsilon=1e-6):
    # Eigenvalue decomposition
    eigenvalues, eigenvectors = torch.linalg.eigh(matrix, UPLO='U')  # 'U' for upper triangular part
    # Clip eigenvalues to ensure they are positive or greater than epsilon
    clipped_eigenvalues = torch.clamp(eigenvalues, min=epsilon)
    # Reconstruct the matrix with modified eigenvalues
    modified_matrix = eigenvectors @ torch.diag(clipped_eigenvalues) @ eigenvectors.T
    return modified_matrix

In [31]:
modified_V = make_positive_definite(normal_V)

In [32]:
dist = torch.distributions.MultivariateNormal(mu_C, modified_V)

In [42]:
Z0 = dist.sample()

In [45]:
print(Z0.shape[0])

3840


In [69]:
def draw_independent_samples(mean, covariance, num_blocks, num_samples=1):
    # Get the size of each block
    block_size = mean.shape[0] // num_blocks
    
    # Reshape the mean vector to have shape (num_blocks, block_size)
    mean = mean.view(num_blocks, block_size)
    
    # Draw independent samples for each block
    samples = []
    for i in range(num_blocks):
        block_mean = mean[i]  # Extract mean for each block
        block_covariance = make_positive_definite(covariance[i * block_size : (i + 1) * block_size, i * block_size : (i + 1) * block_size])
        
        # Create a MultivariateNormal distribution for the block
        mvn_distribution = torch.distributions.MultivariateNormal(block_mean, block_covariance)
        
        # Draw samples from the distribution
        block_samples = mvn_distribution.sample((num_samples,))
        samples.append(block_samples)
    
    # Stack the samples along the last dimension
    samples = torch.cat(samples, dim=1)
    
    return samples[0]

In [70]:
Z = draw_independent_samples(mu_C, normal_V, n)
print(Z)

tensor([ 9.6707e+05,  9.0367e-02, -7.6190e+01,  ..., -1.9676e+00,
        -9.4575e+00,  1.9471e+01], dtype=torch.float64)


In [34]:
Z1 = Z0-help@np.linalg.inv(B.T@help)@B.T@(Z0-Cs)
print(Z1.shape)

(3840,)


In [35]:
Delta = np.identity(q_hat+1)*0.01**2
epsilon = Priors.multivariate_normal_prior(Y0, Delta)[0]
e = np.repeat(epsilon, n)

In [36]:
Cs = Z1 - V@ws@(ws.T@V@ws+Delta)@ws.T@(Z1-e)
print(Cs.shape)
CC = np.reshape(Cs, (n, q_hat+1))
print(CC.shape)
XX = CC+F
print(X_i.shape)
print(XX.shape)

(3840,)
(120, 32)
(120, 32)
(120, 32)


### Step 2: Calculate $(F, S_m)$:

Calculate $\mu_F$ and $\Sigma_F$:

In [37]:
i_2 = np.zeros(q_hat+1)
i_2[1] = 1

mu_F = i_1*f_0+i_2*mu_m
print(mu_F.shape)

(32,)


In [38]:
A, Sigma_A = Initialize.init_Sigma_A(R_hat, T, q_hat, s_Da)
Sigma_F = sigma_m**3*Sigma_m+s_Da**2*Sigma_A
print(Sigma_F.shape)

(32, 32)


Calculate $e$:

In [39]:
e = np.kron(np.ones(n).T, I).T
print(e.shape)

(3840, 32)


Calculate $m_F$, $V_F$, and draw $F-\mu_F$:

In [40]:
V_F = np.linalg.inv(np.linalg.inv(Sigma_F)+e.T@np.linalg.inv(Sigma)@e+np.linalg.inv(Delta))
print(V_F.shape)

(32, 32)


In [41]:
m_F = V_F@(e.T@np.linalg.inv(Sigma)@(X-mu_C-e@mu_F)+np.linalg.inv(Delta)@((ws.T@X)-Y0-mu_F))
print(m_F.shape)

TypeError: unsupported operand type(s) for -: 'numpy.ndarray' and 'Tensor'

In [ ]:
draw = Priors.multivariate_normal_prior(m_F, V_F)[0]
FF = draw+mu_F
print(FF.shape)

(32,)


Calculate $\Sigma_S$ and draw $S_m$:

In [ ]:
Sigma_S = sigma_m**2*Sigma_m
print(Sigma_S.shape)
mean_S_m = Sigma_S@np.linalg.inv(Sigma_F)@(FF-mu_F)
var_S_m = Sigma_S-Sigma_S@np.linalg.inv(Sigma_F)@Sigma_S

SS_m = Priors.multivariate_normal_prior(mean_S_m, var_S_m)[0]

(32, 32)
